# NLP XDD

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
#model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

c:\Users\KHYLE MATTHEW\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
dataset = load_dataset("QuyenAnhDE/Diseases_Symptoms")
dataset["train"][100]

c:\Users\KHYLE MATTHEW\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


{'Code': 101,
 'Name': 'Postpartum Depression',
 'Symptoms': 'Persistent feelings of sadness, emptiness, or hopelessness, frequent crying, irritability, loss of interest or pleasure in activities, difficulty bonding with the baby, changes in appetite or sleep patterns, overwhelming fatigue, feelings of guilt or worthlessness, trouble concentrating or making decisions, thoughts of self-harm or harming the baby',
 'Treatments': 'Psychotherapy (such as cognitive-behavioral therapy or interpersonal therapy), support groups or peer counseling, antidepressant medication (if recommended by a healthcare professional), hormone therapy (in some cases), lifestyle changes (such as exercise, healthy diet, sleep hygiene), involving a partner or family in the care of the baby, social support and understanding from family and friends, regular follow-up and monitoring by healthcare professionals'}

In [3]:
dataset["train"].features

{'Code': Value(dtype='int64', id=None),
 'Name': Value(dtype='string', id=None),
 'Symptoms': Value(dtype='string', id=None),
 'Treatments': Value(dtype='string', id=None)}

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["Symptoms"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [26]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(400))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(400))

In [18]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("medicalai/ClinicalBERT")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="/test_trainer")

In [20]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [21]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


In [23]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics,
)

In [29]:
trainer.train()

AttributeError: 'AcceleratorState' object has no attribute 'distributed_type'